In [4]:
!pip install keras-tuner

In [5]:
import tensorflow as tf 
from tensorflow import keras 
import numpy as np 

In [6]:
fashion_mnist=keras.datasets.fashion_mnist

In [7]:
(train_images,train_labels),(test_images,test_labels)=fashion_mnist.load_data()

In [8]:
train_images=train_images/255.0
test_images=test_images/255.0

In [9]:
train_images[0].shape

(28, 28)

In [10]:
train_images=train_images.reshape(len(train_images),28,28,1)
test_images=test_images.reshape(len(test_images),28,28,1)

In [11]:
def build_model(hp):
  model = keras.Sequential([
   keras.layers.Conv2D(
       filters=hp.Int('conv_1_filter',min_value=32,max_value=128, step=16),
       kernel_size=hp.Choice('conv_1_kernel', values =[3,5]),
       activation='relu',
       input_shape=(28,28,1)  
   ),
   keras.layers.Conv2D(
       filters=hp.Int('conv_2_filter', min_value=32, max_value=64, step=16),
       kernel_size=hp.Choice('conv_2_kernel', values =[3,5]),
       activation='relu',
   ),
   keras.layers.Flatten(),
   keras.layers.Dense(
       units=hp.Int('dense_1_units', min_value=32, max_value=128, step=16),
       activation='relu'
   ),
   keras.layers.Dense(10, activation='softmax')#output layer 
  ])
   
  model.compile(optimizer=keras.optimizers.Adam(hp.Choice('learning_rate',values=[1e-2, 1e-3])),
                 loss='sparse_categorical_crossentropy',
                 metrics=['accuracy'])   
   
  return model               

In [12]:
from kerastuner import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameters

In [13]:
tuner_search=RandomSearch(build_model,
                          objective='val_accuracy',
                          max_trials=5,directory='output',project_name="Mnist Fashion")

INFO:tensorflow:Reloading Oracle from existing project output/Mnist Fashion/oracle.json
INFO:tensorflow:Reloading Tuner from output/Mnist Fashion/tuner0.json


In [14]:
tuner_search.search(train_images,train_labels,epochs=3,validation_split=0.1)

Trial 4 Complete [00h 17m 22s]
val_accuracy: 0.8433333039283752

Best val_accuracy So Far: 0.9125000238418579
Total elapsed time: 00h 58m 30s
INFO:tensorflow:Oracle triggered exit


In [17]:

model=tuner_search.get_best_models(num_models=1)[0]

In [18]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 26, 26, 112)       1120      
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 24, 24, 48)        48432     
_________________________________________________________________
flatten (Flatten)            (None, 27648)             0         
_________________________________________________________________
dense (Dense)                (None, 96)                2654304   
_________________________________________________________________
dense_1 (Dense)              (None, 10)                970       
Total params: 2,704,826
Trainable params: 2,704,826
Non-trainable params: 0
_________________________________________________________________
